In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings('ignore')
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import LabelEncoder

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

In [2]:
pd.set_option('display.max_rows', 50)

In [3]:
df = pd.read_csv('data_sorted.csv')

In [4]:
#filtering out useless data
df = df[df.state != 'unavailable']
df = df[df.state != 'caution-Conflicting-Traffic']

In [5]:
#grouping of signalgroups
signalgroups = [d for _, d in df.groupby('id')]

In [6]:
#resetting index
signalgroups = [d.reset_index() for d in signalgroups]

In [7]:
def tijd(signalgroup):
    #make a seperate indices column for later use
    df1 = signalgroup.loc[(signalgroup.ischanged == True)]
    df1["time_shift"] = df1["state_timestamp"].shift(-1).replace()
    lijst = df1['time_shift'].to_list()
    indeces = df1['time_shift'].index.to_list()
    # iterate through all values per signalgroup
    for value in range(len(lijst)-1):
        begin = indeces[value]
        end = indeces[value + 1]
        signalgroup.loc[begin:end, 'time_s'] = lijst[value]
    return signalgroup

In [8]:

id_list = []
groups = []
minmaxgroups = []
for signalgroup in tqdm(signalgroups):
    # make a list with all signal group IDs for later use
    id_list.append(signalgroup['id'].to_list()[0])
    
    # put every signalgroup through the tijd() function
    signalgroup['state'] = np.where( ( (signalgroup['state'] == 'permissive-clearance')| (signalgroup['state'] == 'caution-Conflicting-Traffic') | (signalgroup['state'] == 'protected-Movement-Allowed') | (signalgroup['state'] == 'permissive-Movement-Allowed') | (signalgroup['state'] == 'pre-Movement') | (signalgroup['state'] == 'protected-clearance') ), "green", "red")
    signalgroup["ischanged"] = signalgroup["state"].shift(1, fill_value=signalgroup["state"].head(1)) != signalgroup["state"]
    signalgroup = tijd(signalgroup)
    minmax = signalgroup
    minmaxgroups.append(minmax)
    signalgroup = signalgroup.dropna(subset=['state_end_likely'])
    #signalgroup = signalgroup.loc[~((signalgroup['ischanged']== False) & (signalgroup['state_end_likely']== pd.NaT))]
    signalgroup["new_sequence"] = signalgroup["time_s"].shift(1) != signalgroup["time_s"]
    groups.append(signalgroup) 



100%|██████████| 42/42 [00:16<00:00,  2.48it/s]


In [9]:
#resetting index
groups = [d.reset_index() for d in groups]
df = pd.concat(groups[:7], ignore_index=True)
df.dropna()

,level_0,index,received_timestamp,id,state_timestamp,state,state_end_min,state_end_max,state_end_likely,state_end_confidence,state_exceptional_condition,ischanged,time_s,new_sequence


In [10]:
# define train and test set
x = df[['received_timestamp', 'state_timestamp', 'state', 'id']]
le = LabelEncoder()
x['state'] = le.fit_transform(x['state'])
x['id'] = le.fit_transform(x['id'])
y = df['time_s']
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [11]:
# train model
mlp = MLPRegressor()
mlp.fit(x_train, y_train)

MLPRegressor()

In [12]:
# get the predictions and score
pred = mlp.predict(x_test)
mlp.score(x_test, y_test)

0.9847173618625223

In [13]:
# a random data point
x_test.iloc[1]

received_timestamp    1654898003550
state_timestamp       1654898003457
state                             0
id                                2
Name: 7314, dtype: int64

In [14]:
# it's real time change
y_test.iloc[1]

1654898005312.0

In [15]:
# the predicted time change
pred[1]

1654895851891.3115